# Sauim bicolor Vocalization Classifier and Analysis

This notebook demonstrates how to use the Perch bird vocalization classifier model from TensorFlow Hub to extract embeddings from audio files of Sauim bicolor primate and perform analysis on the extracted embeddings, such as hierarchical clustering.

The notebook covers the following steps:

1.  **Setup**: Installing necessary libraries and loading the Perch model.
2.  **Data Loading**: Downloading example audio files and a CSV file containing labels.
3.  **Feature Extraction**: Defining and using a function to extract embeddings and spectrograms from audio segments.
4.  **Spectrogram Visualization**: Plotting spectrograms to visualize the audio data.
5.  **Clustering Analysis**: Performing hierarchical clustering on the extracted embeddings and visualizing the results as a dendrogram.

In [ ]:
%pip install tensorflow_hub -q

In [ ]:
import csv
import librosa
import gdown
import zipfile
import shutil
import os
import soundfile as sf
from scipy import signal
from scipy.io import wavfile
from tqdm.notebook import trange, tqdm
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import LabelEncoder
import tensorflow_hub as hub
import tensorflow as tf
tf.experimental.numpy.experimental_enable_numpy_behavior()

In [ ]:
# Remove the directory and all its contents
if os.path.exists('embedding_vectors'):
    shutil.rmtree('embedding_vectors')
    print("Folder 'embedding_vectors' removed.")

if os.path.exists('Parque do Mindú'):
    shutil.rmtree('Parque do Mindú')
    print("Folder 'Parque do Mindú' removed.")

if os.path.exists('MinduPark_38'):
    shutil.rmtree('MinduPark_38')
    print("Folder 'MinduPark_38' removed.")

if os.path.exists('Parque_do_Mindu_without_Sauim'):
    shutil.rmtree('Parque_do_Mindu_without_Sauim')
    print("Folder 'Parque_do_Mindu_without_Sauim' removed.")

if os.path.exists('model'):
    shutil.rmtree('model')
    print("Folder 'model' removed.")

Automatically download the necessary files

In [ ]:
!wget https://github.com/juancolonna/Sauim/raw/main/records/records.csv -O records.csv
!wget https://github.com/juancolonna/Sauim/raw/main/records/Sauim.wav -O Sauim.wav
!wget https://github.com/juancolonna/Sauim/raw/main/records/Anurans.wav -O Anurans.wav
!wget https://github.com/juancolonna/Sauim/raw/main/records/Anthrophony.wav -O Anthrophony.wav
!wget https://github.com/juancolonna/Sauim/raw/main/records/Background.wav -O Background.wav
!wget https://github.com/juancolonna/Sauim/raw/main/records/Birds.wav -O Birds.wav
!wget https://github.com/juancolonna/Sauim/raw/main/records/Geophony.wav -O Geophony.wav
!wget https://github.com/juancolonna/Sauim/raw/main/records/Mindu_Saguinus%20bicolor_02.02.19-000.wav -O Mindu_Saguinus_bicolor_02.02.19-000.wav

!wget https://github.com/juancolonna/Sauim/raw/main/records/Parque_do_Mindú.zip -O Parque_do_Mindú.zip
!unzip -q Parque_do_Mindú.zip

!wget https://github.com/juancolonna/Sauim/raw/main/records/Parque_do_Mindu_38.zip -O Parque_do_Mindu_38.zip
!unzip -q Parque_do_Mindu_38.zip -d Parque_do_Mindu_38

!wget https://github.com/juancolonna/Sauim/raw/main/perch_v8.zip -O perch_v8.zip
!unzip -o perch_v8.zip -d ./model

In [ ]:
# it is preferible to load the frozen model from github as follow
model = hub.load("./model")  # path containing saved_model.pb

# If it presents any error you can load Perch model from kaggle as follow:
# model = hub.load('https://www.kaggle.com/models/google/bird-vocalization-classifier/TensorFlow2/bird-vocalization-classifier/8')

In [ ]:
def extract_embeddings(file: str, wlen: int, hop_len: int, bandpass_filter: bool):
    """
    Extracts embedding vectors and spectrograms from an audio file.

    Args:
        file (str): Path to the audio file.
        wlen (int): Window length in samples.
        hop_len (int): Hop length in samples.

    Returns:
        tuple: (list of embedding vectors, list of spectrogram arrays)

    Example:
        >>> sr = 32000  # Sampling rate in Hz
        >>> window_len = int(5 * sr)  # 5-second window
        >>> emb, specs = extract_embeddings('Sauim.wav', wlen=window_len, hop_len=window_len)
        >>> np.save('sauim_embedding_vectors.npy', emb)
        >>> np.save('sauim_spectrograms.npy', specs)
    """

    y, sr = librosa.load(file, sr=32000)  # Load audio
    y = y / np.max(np.abs(y)) # Normalize amplitude

    if bandpass_filter:
        # 2) Design a band-pass Butterworth (4th order). Nyquist = fs/2, SciPy handles normalization with fs=
        sos = signal.butter(N=4, Wn=[5000.0, 10000.0], btype="bandpass", fs=sr, output="sos")
        # 3) Zero-phase filtering (filtfilt) along the time axis (axis=0 for (samples, channels))
        y = signal.sosfiltfilt(sos, y.astype(np.float32), axis=0)
        wavfile.write(file[:-4]+'_filtered.wav', sr, y.astype(np.float32))

    embedding_vectors = []
    spectrograms = []

    for time in tqdm(range(0, len(y), hop_len)):
        if time + wlen < len(y):
            segment = y[time:time+wlen]
            model_outputs = model.infer_tf(segment[np.newaxis, :])
            embedding_vectors.append(model_outputs['embedding'].numpy()[0])
            spectrograms.append(model_outputs['frontend'][0].T.numpy())

    return embedding_vectors, spectrograms

def plot_spectrogram(species: str, spectrograms):
    """
    Plots a Mel spectrogram in dB scale for a given species.

    Args:
        species (str): Species name.
        spectrograms (ndarray): Spectrogram array.
        sr (int): Sampling rate in Hz.

    Example:
        >>> sr = 32000
        >>> window_len = int(5 * sr)
        >>> emb, specs = extract_embeddings('Sauim.wav', wlen=window_len, hop_len=window_len)
        >>> # Plot the first extracted spectrogram
        >>> plot_spectrogram("Saguinus bicolor", specs[0], sr)
    """
    S_dB = librosa.power_to_db(spectrograms, ref=np.max) # Convert to decibels

    plt.figure(figsize=(10, 3))
    librosa.display.specshow(S_dB, sr=32000, x_axis='time', y_axis='mel',
                            hop_length=256+64, # Provide hop_length for correct time axis
                            cmap='viridis') # Ensure origin is lower for standard freq plot

    plt.colorbar(format='%+2.0f dB')
    plt.title(f'Spectrogram of a 5-second segment of {species}')
    plt.xlabel('Time (s)')
    plt.ylabel('Mel Frequency (Hz)') # Librosa will automatically format this as Mel-Hz

    plt.tight_layout()
    plt.show()

Extract embeddings and spectrograms for a 'Sauim.wav' audio file and save them as NumPy arrays.

In [ ]:
os.makedirs("embedding_vectors", exist_ok=True)  # cria se não existir

sr = 32000 # Sampling rate (Hz)
window_len = int(5*sr) # 5-second window length in samples

In [ ]:
# Extract embeddings and spectrograms for each 5-second segment
sauim_vectors, sauim_spectrograms = extract_embeddings('Sauim.wav',
                                                     wlen=window_len,
                                                     hop_len=window_len,
                                                     bandpass_filter=False)

# Save outputs to .npy files
np.save('embedding_vectors/sauim_vectors.npy', sauim_vectors)

# Extract embeddings and spectrograms for each 5-second segment
sauim_vectors_filtered, _ = extract_embeddings('Sauim.wav',
                                                     wlen=window_len,
                                                     hop_len=window_len,
                                                     bandpass_filter=True)

# Save outputs to .npy files
np.save('embedding_vectors/sauim_vectors_filtered.npy', sauim_vectors_filtered)

Background noise segments for anomaly data

In [ ]:
background_vectors, background_spectrograms = extract_embeddings('Background.wav',
                                                     wlen=window_len,
                                                     hop_len=window_len,
                                                           bandpass_filter=False)

np.save('embedding_vectors/background_vectors.npy', background_vectors)

# Extract embeddings and spectrograms for each 5-second segment
background_vectors_filtered, _ = extract_embeddings('Background.wav',
                                                     wlen=window_len,
                                                     hop_len=window_len,
                                                     bandpass_filter=True)

# Save outputs to .npy files
np.save('embedding_vectors/background_vectors_filtered.npy', background_vectors_filtered)

Anuran calls used as negative class

In [ ]:
anurans_vectors, anurans_spectrograms = extract_embeddings('Anurans.wav',
                                                     wlen=window_len,
                                                     hop_len=int(window_len/2),
                                                     bandpass_filter=False)

np.save('embedding_vectors/anurans_vectors.npy', anurans_vectors)

# Extract embeddings and spectrograms for each 5-second segment
anurans_vectors_filtered, _ = extract_embeddings('Anurans.wav',
                                                     wlen=window_len,
                                                     hop_len=int(window_len/2),
                                                     bandpass_filter=True)

# Save outputs to .npy files
np.save('embedding_vectors/anurans_vectors_filtered.npy', anurans_vectors_filtered)

Bird calls used as negative class

In [ ]:
birds_vectors, birds_spectrograms = extract_embeddings('Birds.wav',
                                                     wlen=window_len,
                                                     hop_len=window_len,
                                                     bandpass_filter=False)

np.save('embedding_vectors/birds_vectors.npy', birds_vectors)

# Extract embeddings and spectrograms for each 5-second segment
birds_vectors_filtered, _ = extract_embeddings('Birds.wav',
                                                     wlen=window_len,
                                                     hop_len=window_len,
                                                     bandpass_filter=True)

# Save outputs to .npy files
np.save('embedding_vectors/birds_vectors_filtered.npy', birds_vectors_filtered)

A soundscape record used for new sauim detections

In [ ]:
step = 1
hop_len = int(step*sr) # emulates an sliding window of 5 sec length and 1 sec step

soundscape_vectors, soundscape_spectrograms = extract_embeddings('Mindu_Saguinus_bicolor_02.02.19-000.wav',
                                                     wlen=window_len,
                                                     hop_len=hop_len,
                                                     bandpass_filter=False)

np.save('embedding_vectors/soundscape_vectors.npy', soundscape_vectors)

soundscape_vectors_filtered, soundscape_spectrograms_filtered = extract_embeddings('Mindu_Saguinus_bicolor_02.02.19-000.wav',
                                                     wlen=window_len,
                                                     hop_len=hop_len,
                                                     bandpass_filter=True)

np.save('embedding_vectors/soundscape_vectors_filtered.npy', soundscape_vectors_filtered)

In [ ]:
step = 2
hop_len = int(step*sr) # emulates an sliding window of 5 sec length and 1 sec step

anthrophony_vectors, anthrophony_spectrograms = extract_embeddings('Anthrophony.wav',
                                                     wlen=window_len,
                                                     hop_len=hop_len,
                                                     bandpass_filter=False)

np.save('embedding_vectors/anthrophony_vectors.npy', anthrophony_vectors)

# Extract embeddings and spectrograms for each 5-second segment
anthrophony_vectors_filtered, _ = extract_embeddings('Anthrophony.wav',
                                                     wlen=window_len,
                                                     hop_len=hop_len,
                                                     bandpass_filter=True)

# Save outputs to .npy files
np.save('embedding_vectors/anthrophony_vectors_filtered.npy', anthrophony_vectors_filtered)

In [ ]:
step = 2
hop_len = int(step*sr) # emulates an sliding window of 5 sec length and 1 sec step

geophony_vectors, geophony_spectrograms = extract_embeddings('Geophony.wav',
                                                     wlen=window_len,
                                                     hop_len=hop_len,
                                                     bandpass_filter=False)

np.save('embedding_vectors/geophony_vectors.npy', geophony_vectors)

# Extract embeddings and spectrograms for each 5-second segment
geophony_vectors_filtered, _ = extract_embeddings('Geophony.wav',
                                                     wlen=window_len,
                                                     hop_len=hop_len,
                                                     bandpass_filter=True)

# Save outputs to .npy files
np.save('embedding_vectors/geophony_vectors_filtered.npy', geophony_vectors_filtered)

In [ ]:
!zip -r embedding_vectors.zip embedding_vectors

## Audio files for the Ocupancy model

In [ ]:
os.makedirs("MinduPark", exist_ok=True)  # cria se não existir

step = 2
hop_len = int(step*sr) # emulates an sliding window of 5 sec length and 1 sec step

for i, file in enumerate(os.listdir('Parque do Mindú')):
    MinduPark_vectors, _ = extract_embeddings("Parque do Mindú/"+file,
                                                        wlen=window_len,
                                                        hop_len=hop_len,
                                                        bandpass_filter=False)

    np.save('MinduPark/MinduPark_vectors_'+str(i)+'.npy', MinduPark_vectors)

    MinduPark_vectors_filtered, _ = extract_embeddings("Parque do Mindú/"+file,
                                                        wlen=window_len,
                                                        hop_len=hop_len,
                                                        bandpass_filter=True)

    np.save('MinduPark/MinduPark_vectors_filtered_'+str(i)+'.npy', MinduPark_vectors_filtered)

!zip -r MinduPark.zip MinduPark

In [ ]:
!unzip -q Parque_do_Mindu_without_Sauim.zip

In [ ]:
os.makedirs("MinduPark_without_sauim", exist_ok=True)  # cria se não existir

sr = 32000 # Sampling rate (Hz)
window_len = int(5*sr) # 5-second window length in samples

step = 2
hop_len = int(step*sr) # emulates an sliding window of 5 sec length and 1 sec step

for i, file in enumerate(os.listdir('Parque_do_Mindu_without_Sauim')):
    MinduPark_vectors, _ = extract_embeddings("Parque_do_Mindu_without_Sauim/"+file,
                                                        wlen=window_len,
                                                        hop_len=hop_len,
                                                        bandpass_filter=False)

    np.save('MinduPark_without_sauim/MinduPark_vectors_'+str(i)+'.npy', MinduPark_vectors)

    MinduPark_vectors_filtered, _ = extract_embeddings("Parque_do_Mindu_without_Sauim/"+file,
                                                        wlen=window_len,
                                                        hop_len=hop_len,
                                                        bandpass_filter=True)

    np.save('MinduPark_without_sauim/MinduPark_vectors_filtered_'+str(i)+'.npy', MinduPark_vectors_filtered)

!zip -r MinduPark_without_sauim.zip MinduPark_without_sauim

In [ ]:
os.makedirs("MinduPark_38", exist_ok=True)  # cria se não existir

step = 1
hop_len = int(step*sr) # emulates an sliding window of 5 sec length and 1 sec step

for i, file in enumerate(os.listdir('Parque_do_Mindu_38/Parque do Mindú')):
    MinduPark_vectors, _ = extract_embeddings("Parque_do_Mindu_38/Parque do Mindú/"+file,
                                                        wlen=window_len,
                                                        hop_len=hop_len,
                                                        bandpass_filter=False)

    np.save('MinduPark_38/MinduPark_vectors_'+str(i)+'.npy', MinduPark_vectors)

    MinduPark_vectors_filtered, _ = extract_embeddings("Parque_do_Mindu_38/Parque do Mindú/"+file,
                                                        wlen=window_len,
                                                        hop_len=hop_len,
                                                        bandpass_filter=True)

    np.save('MinduPark_38/MinduPark_vectors_filtered_'+str(i)+'.npy', MinduPark_vectors_filtered)

!zip -r MinduPark_38.zip MinduPark_38

Load labels, compute cosine distance matrix from embedding vectors, perform hierarchical clustering, and plot a dendrogram.

In [ ]:
# 1. Load labels from CSV
records = pd.read_csv('records.csv')  # adjust path if needed
labels = records['Bosque_S.bicolor_12.4.19-000'].tolist()

# Encode labels as integers
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)

# 2. Compute cosine distance matrix (condensed form)
cosine_distances = pdist(sauim_vectors, metric='cosine')

# 3. Perform hierarchical clustering
Z = linkage(cosine_distances, method='ward')  # try 'ward', 'complete', etc.

# 4. Plot dendrogram
plt.figure(figsize=(20, 5))
dendrogram(Z, labels=encoded_labels, leaf_font_size=10)
plt.xlabel("Records segments with sauim vocalizations")
plt.ylabel("Cosine distances")
plt.tight_layout()
plt.savefig("dendongram.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
for i in zip(encoded_labels,labels):
    print(i)

In [ ]:
# 1. Load labels from CSV
records = pd.read_csv('records.csv')  # adjust path if needed
labels = records['Bosque_S.bicolor_12.4.19-000'].tolist()

# Encode labels as integers
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)

# 2. Compute cosine distance matrix (condensed form)
cosine_distances = pdist(sauim_vectors_filtered, metric='cosine')

# 3. Perform hierarchical clustering
Z = linkage(cosine_distances, method='ward')  # try 'ward', 'complete', etc.

# 4. Plot dendrogram
plt.figure(figsize=(20, 5))
dendrogram(Z, labels=encoded_labels, leaf_font_size=10)
plt.xlabel("Records segments with sauim vocalizations")
plt.ylabel("Cosine distances")
plt.tight_layout()
plt.savefig("dendongram_filtered.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()